Based on notebook 6, for further experiments only knn, xgboost and linear svc will be considered.  

Handle class imbalance by selectively removing poorly classified classes. Investigate the impact of individual classes on the classifier's performance and potentially improve the accuracy by removing problematic classes.

#convert reason to numbers  
{'Mau Serviço Prestado': 0,  
'Condições de entrega': 1,   
'Atraso de entrega': 2,   
'Enganos': 3}  

1 and 3 are the problematic classes. Class 3 has less samples than 1 but still has greater recall, so I'm going to try:  
* removing class 1- Condições de Entrega
* removing class 3
* join classes 0,1,3 vs 2  


After removing the class and training a new classifier, I'm going to apply this new classifier on the excluded class and use the predicted labels as new labels to train another classifier. This can be seen as an iterative error analysis approach, where you iteratively refine the classifier by addressing specific errors or problematic cases.

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler #fixed import

import nltk
import re
from langdetect import detect
from scipy.sparse import hstack
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize

In [17]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")
df.head()

,date,title,reason,description,zone,narrative_tfidf,narrative_embeddings,narrative_bert,narrative_tfidf_title,narrative_bert_title,...,tfidf_title,title_tfidf_nolemma,events,keywords,locs,embeddings_title,tfidf_keywords,embeddings_keywords,events_clean,events_tfidf
0,2021-10-11 12:26:32,CTT - Encomenda entregue danificada e com etiq...,0,Bom dia venho por este meio apresentar uma rec...,Lisboa,bom dia venho meio apresentar reclamação ctt e...,bom dia venho por este meio apresentar uma rec...,Bom dia venho por este meio apresentar uma rec...,ctt encomenda entregar danificar etiqueta envi...,ctt - encomenda entregue danificada e com etiq...,...,ctt encomenda entregar danificar etiqueta envi...,ctt encomenda entregue danificada etiquetas en...,"['venho', 'apresentar', 'Enviei', 'destinado',...",apresentar uma reclamação tinha caixa figura e...,"['Israel', 'Aboboda', 'Israel', 'de Braga', 'O...",ctt - encomenda entregue danificada e com etiq...,apresentar reclamação caixa figura envio israe...,apresentar uma reclamação tinha caixa figura e...,venho apresentar enviei destinado feito proteg...,quase destruir figura partida chegar Caixa che...
1,2021-08-31 23:20:15,CTT - Carta com certificado rasgado!,1,- Boa Noite!\r\n\r\n- Venho por este meio recl...,Coimbra,bom noite venho meio reclamar data ás durante ...,- boa noite!\r\n\r\n- venho por este meio recl...,- Boa Noite! Venho por este meio reclamar que ...,ctt carta certificar rasgar bom noite venho me...,ctt - carta com certificado rasgado! - Boa No...,...,ctt carta certificar rasgar,ctt carta certificado rasgado,"['Venho', 'reclamar', 'deparei -me', 'continha...",Lilia Pimentel certificado profissional passag...,['Boa Noite!'],ctt - carta com certificado rasgado !,lilia pimentel certificar profissional passage...,lilia pimentel certificado profissional passag...,venho reclamar deparei -me continha tem,NaN
2,2021-02-01 19:08:32,CTT - Encomenda registada que nunca chegou ao ...,2,"Boa tarde,\r\nNo dia 17 de Dezembro de 2020 en...",Aveiro,bom tarde dia dezembro enviar encomenda França...,"boa tarde,\r\nno dia 00 de dezembro de 0000 en...","Boa tarde, No dia 17 de Dezembro de 2020 envie...",ctt encomenda registar nunca chegar destine o ...,ctt - encomenda registada que nunca chegou ao ...,...,ctt encomenda registar nunca chegar destine o,ctt encomenda registada nunca chegou destino,"['enviei', 'registado', 'vejo', 'chegou', 'sai...",França em correio registado França em correio ...,"['França', 'Roissy', 'ROISSY']",ctt - encomenda registada que nunca chegou ao ...,frança correio registar frança correio encomen...,frança em correio registado frança em correio ...,enviei registado vejo chegou saiu chegou tem p...,resolr
3,2018-04-19 09:04:34,CTT - Correio Verde,2,"Bom dia,\n\nPretendia esclarecer uma situação ...",Lisboa,bom dia pretender esclarecer situação ocorrer ...,"bom dia,\n\npretendia esclarecer uma situação ...","Bom dia, Pretendia esclarecer uma situação que...",ctt correio verde bom dia pretender esclarecer...,"ctt - correio verde Bom dia, Pretendia esclar...",...,ctt correio verde,ctt correio verde,"['Pretendia esclarecer', 'ocorreu', 'desloquei...",Pretendia esclarecer uma situação Pretendia es...,['Lisboa'],ctt - correio verde,pretender esclarecer situação pretender esclar...,pretendia esclarecer uma situação pretendia es...,pretendia esclarecer ocorreu desloquei dado fe...,esclarecer saber semana enviar eventualmente a...
4,2019-07-04 10:20:41,"CTT - Aviso de recepção da carta registada, pr...",2,venho por este meio solicitar a vossa ajuda pa...,Porto,venho meio solicitar vosso ajudar saber onde e...,venho por este meio solicitar a vossa ajuda pa...,venho por este meio solicitar a vossa ajuda pa...,ctt aviso recepção carta registar preciso urge...,"ctt - aviso de recepção da carta registada, pr...",...,ctt aviso recepção carta registar preciso urge...,ctt aviso recepção carta registada preciso urg...,"['venho', 'solicitar', 'saber onde se encontra...",papais para o centro 

In [3]:
df['reason'].unique()

array([0, 1, 2, 3], dtype=int64)

# Remove class 1- condições de entrega

In [137]:
def remove_class_and_train_classifier(dataset, class_to_remove, clf):
    # Filter out the class to remove
    filtered_df = dataset[dataset['reason'] != class_to_remove]
    
    # Separate features and labels
    y = filtered_df['reason']
    X = filtered_df['narrative_tfidf_title']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # Split the remaining data into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)
    
    # Map the class labels to match the expected classes
    # because of error message, the expected classes are [0, 1, 2], but the unique values in y_train are [0, 2, 3]
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)
    
    vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.95, min_df=2)
    X_train_tf = vectorizer.fit_transform(X_train)
    X_val_tf = vectorizer.transform(X_val)
    clf.fit(X_train_tf, y_train_encoded)
    y_val_pred_encoded = clf.predict(X_val_tf)
    
    # Reverse the label encoding to get the original class labels
    y_val_pred = label_encoder.inverse_transform(y_val_pred_encoded)
    
    l=[0,1,2,3]
    l.remove(class_to_remove)
    confusion_mat = confusion_matrix(y_true=y_val, y_pred=y_val_pred, labels=l)
    class_rep= classification_report(y_val, y_val_pred)
    
    print(class_rep)
    print(confusion_mat)
    
    # Create a DataFrame with the original texts, predicted labels, and true labels
    results_df = pd.DataFrame({'narrative_bert_title': dataset.loc[y_val.index, 'narrative_bert_title'],
                               'narrative_tfidf_title': dataset.loc[y_val.index, 'narrative_tfidf_title'],
                               'predicted label': y_val_pred,
                               'true label': y_val})

    misclassified_texts = results_df[results_df['predicted label'] != results_df['true label']]
    
    # Apply the trained classifier on the excluded class
    X_excluded = dataset[dataset['reason'] == class_to_remove]['narrative_tfidf_title']
    X_excluded_tf = vectorizer.transform(X_excluded)
    y_excluded_pred_encoded = clf.predict(X_excluded_tf)
    y_excluded_pred = label_encoder.inverse_transform(y_excluded_pred_encoded)

    excluded_predictions_df = pd.DataFrame({'narrative_bert_title': dataset.loc[dataset['reason'] == class_to_remove, 'narrative_bert_title'],
                                            'narrative_tfidf_title': dataset.loc[dataset['reason'] == class_to_remove, 'narrative_tfidf_title'],
                                            'reason': y_excluded_pred,
                                            'initial Label': dataset.loc[dataset['reason'] == class_to_remove, 'reason']})
    
    return y_val_pred, y_val, misclassified_texts, excluded_predictions_df, filtered_df

In [146]:
def new_classifier(df, clf, class_to_remove):
    # Separate features and labels
    y = df['reason']
    X = df['narrative_tfidf_title']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # Split the remaining data into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)
    
    # Map the class labels to match the expected classes
    # because of error message, the expected classes are [0, 1, 2], but the unique values in y_train are [0, 2, 3]
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)
    
    vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.95, min_df=2)
    X_train_tf = vectorizer.fit_transform(X_train)
    X_val_tf = vectorizer.transform(X_val)
    clf.fit(X_train_tf, y_train_encoded)
    y_val_pred_encoded = clf.predict(X_val_tf)
    
    # Reverse the label encoding to get the original class labels
    y_val_pred = label_encoder.inverse_transform(y_val_pred_encoded)
    
    l=[0,1,2,3]
    l.remove(class_to_remove)
    confusion_mat = confusion_matrix(y_true=y_val, y_pred=y_val_pred, labels=l)
    class_rep= classification_report(y_val, y_val_pred)
    
    print(class_rep)
    print(confusion_mat)

# KNN

###  Misclassified texts

In [123]:
knn_predictions, knn_true_labels, knn_misclassified_texts, knn_predictions_excluded, knn_filtered_df= remove_class_and_train_classifier(df, 1,KNeighborsClassifier())

              precision    recall  f1-score   support

           0       0.62      0.60      0.61      1195
           2       0.72      0.80      0.76      1609
           3       0.46      0.19      0.27       228

    accuracy                           0.68      3032
   macro avg       0.60      0.53      0.55      3032
weighted avg       0.66      0.68      0.67      3032

[[ 720  433   42]
 [ 309 1291    9]
 [ 123   61   44]]


In [124]:
knn_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
7448,"Boa tarde, venho por este meio informar a negl...",bom tarde venho meio informar negligencia part...,3,0
14819,ctt - correspondência atrasada Comprei um aut...,ctt correspondência atrasar comprar automóvel ...,0,2
9571,ctt - reclamação de objecto entregue Ao quest...,ctt reclamação objecto entregar questionar Cen...,2,0
5576,"boa tarde, desde 6f dia 3 de julho que estou a...",bom tarde desde dia julho esperar encomenda re...,0,2
7340,ctt - os ctt dão-se ao luxo de brincar com os ...,ctt ctt luxo brincar cliente encomenda disposi...,2,0
...,...,...,...,...
6615,ctt - cartas que chegam 3 meses depois Venho ...,ctt carta chegar mês venho aqui indicar descon...,0,2
11575,ctt - encomenda não entregue Tenho uma encome...,ctt encomenda entregar encomenda desde julho v...,2,0
281,"Bom dia, gostaria de saber como podem resolver...",bom dia gostar saber poder resolver questão en...,0,3
5580,Tenho a seguinte encomenda RN758511072GB que t...,seguinte encomenda status entregar dia gostar ...,0,3


In [126]:
print('MISCLASSIFIED KNN TEXTS')
for i in range(100):
    print(knn_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(knn_misclassified_texts['true label'].iloc[i],
                                                                knn_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED KNN TEXTS
Boa tarde, venho por este meio informar a negligencia por parte dos colaboradores do ctt correios de delães/ Vila nova de Famalicão, a má entrega das cartas das respetivas pessoas. Não sendo a primeira vez que me desloco ao balcão da mesma para entrega de cartas as quais não me são pretensidas. Informo que a minha caixa de correio se encontra respetivamente identificada com o meu nome e o da minha esposa, é não com o nome de terceiros. Sem mais nada acrescentar agradeço que verifiquem a situação e a resolvam pois o incomodo ja é o bastante. Cumprimentos Hugo Araújo 
TRUE LABEL IS 0 BUT IT WAS CLASSIFIED AS 3


ctt - correspondência atrasada  Comprei um automóvel já a 02 meses, foi feito a transferência e o documento não chega. Já entrei em contato com conservatória de automóveis e disseram que a transferência já foi efetuada e que a demora é por parte do ctt. Aguardo resposta pois preciso do carro para trabalhar. Obrigado 
TRUE LABEL IS 2 BUT IT WAS CLASSIFIED A

### Apply classifier to the excluded class

In [127]:
knn_predictions_excluded

,narrative_bert_title,narrative_tfidf_title,reason,initial Label
1,ctt - carta com certificado rasgado! - Boa No...,ctt carta certificar rasgar bom noite venho me...,2,1
19,"ctt - encomendas no chão Bom dia, é inadmissí...",ctt encomenda chão bom dia inadmissível cartei...,0,1
24,Fiz uma compra online em que viria uma encomen...,fiz compra online vir encomenda país enviar no...,0,1
26,ctt - encomenda não entregue Foi me enviada u...,ctt encomenda entregar enviar encomenda passad...,0,1
36,ctt - envio indevido para apresentação à alfân...,ctt envio indever apresentação alfândegar exmo...,2,1
...,...,...,...,...
22355,ctt - encomenda não entregue Venho por este m...,ctt encomenda entregar venho meio fazer reclam...,2,1
22359,ctt - cobrança das taxas já pagas no site Bom...,ctt cobrança taxa pagas site bom tarde dia fiz...,2,1
22368,ctt - tentativa falhada de levantar a minha en...,ctt tentativa falhada levantar encomenda bom n...,0,1
22370,fiz uma compra pela Internet na qual existia 3...,fiz compra Internet existir artigo chegar eu o...,2,1


In [128]:
print('PREDICTIONS OF KNN ON EXCLUDED TEXTS')
for i in range(100):
    print(knn_predictions_excluded['narrative_bert_title'].iloc[i])
    print('INITIAL LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(knn_predictions_excluded['initial Label'].iloc[i],
                                                                knn_predictions_excluded['reason'].iloc[i]))
    print('\n')

PREDICTIONS OF KNN ON EXCLUDED TEXTS
ctt - carta com certificado rasgado!  - Boa Noite! - Venho por este meio reclamar que na data 31/08/2021 ás 10:00h, durante a passagem do correio deparei-me com uma carta que continha um documento de extrema importância (certificado profissional) com um rasgo. - Dito isto, este documento no seu estado não tem valor. - Obrigada! - Com os maiores cumprimentos: Lilia Pimentel 
INITIAL LABEL IS 1 BUT IT WAS CLASSIFIED AS 2


ctt - encomendas no chão  Bom dia, é inadmissível que o carteiro em vez de colocar as encomendas na caixa de correio as mande para dentro de nossa casa como se de um jornal se tratasse e vivêssemos num país americano! Para além de estar um dia de muita chuva onde danificou bastante as mesmas o meu cão anda pela casa mas quando vê objetos novos brinca com os mesmos e os destrói como foi o caso. E o carteiro que não me venha dizer que não conseguia colocar na caixa porque quando isso acontece deixasse um papel para ir levantar a encom

### Use predicted classes as new labels to train new classifier

In [147]:
# Concatenate the excluded predictions dataframe with the original dataset, keeping only the specified columns
new_dataset = pd.concat([knn_filtered_df[['reason', 'narrative_tfidf_title','narrative_bert_title']], 
                         knn_predictions_excluded[['reason', 'narrative_tfidf_title','narrative_bert_title']]], ignore_index=True)


shuffled_dataset = new_dataset.sample(frac=1, random_state=42)
shuffled_dataset

,reason,narrative_tfidf_title,narrative_bert_title
6769,0,ctt reclamação dia dezembro enviar encomenda c...,ctt - reclamação No dia 9 de dezembro enviei ...
8171,2,ctt atraso correspondência reformar doença sem...,ctt - atraso da correspondência Sou reformada...
6603,0,ctt reclamação carteiro venho poe meio reclama...,ctt - reclamação para com o carteiro Venho po...
5305,0,ctt anulação pagar resolvido fiz envio estrang...,"ctt - anulação paga, mas não resolvida Fiz um..."
20251,2,ctt problema receber encomenda sinceramente nu...,ctt - problemas em receber minhas encomendas ...
...,...,...,...
11964,0,ctt irresponsabilidade negligência falta respe...,"ctt - irresponsabilidade, negligência e falta ..."
21575,0,ctt correspondência abrir venho meio apresenta...,ctt - correspondência aberta Venho por este m...
5390,2,ctt alfândegar féria dar entrada dia julho alf...,ctt - alfândega de férias Deu entrada no dia ...
860,2,ctt remessa pedir desalfandegamento recebi pre...,ctt - remessa com pedido de desalfandegamento ...


In [148]:
new_classifier(shuffled_dataset, KNeighborsClassifier(), 1)

              precision    recall  f1-score   support

           0       0.65      0.63      0.64      1402
           2       0.72      0.79      0.75      1720
           3       0.54      0.25      0.34       242

    accuracy                           0.68      3364
   macro avg       0.64      0.56      0.58      3364
weighted avg       0.68      0.68      0.68      3364

[[ 881  484   37]
 [ 347 1359   14]
 [ 124   57   61]]


# XGBoost

###  Misclassified texts

In [139]:
xgb_predictions, xgb_true_labels, xgb_misclassified_texts, xgb_predictions_excluded, xgb_filtered_df= remove_class_and_train_classifier(df, 1,XGBClassifier())

              precision    recall  f1-score   support

           0       0.70      0.66      0.68      1195
           2       0.76      0.85      0.80      1609
           3       0.55      0.25      0.34       228

    accuracy                           0.73      3032
   macro avg       0.67      0.59      0.61      3032
weighted avg       0.72      0.73      0.72      3032

[[ 784  376   35]
 [ 225 1373   11]
 [ 117   55   56]]


In [140]:
xgb_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
13825,ctt - serviço siga Recebi o vale postal para ...,ctt serviço siga recebi valer postal levantame...,0,2
7340,ctt - os ctt dão-se ao luxo de brincar com os ...,ctt ctt luxo brincar cliente encomenda disposi...,2,0
10951,"Exmos. Senhores, Resido na Ilha das Flores, na...",exmos senhor reser ilha flor freguesia ponto d...,0,2
13725,ctt - custo alfandega - discriminação da mesma...,ctt custo alfandega discriminação mesmo venho ...,3,0
17163,ctt - encomendas que desaparecem no centro dis...,ctt encomenda desaparecer Centro distribuição ...,0,2
...,...,...,...,...
18508,Dia 23 Maio foi enviado uma encomenda a cobran...,dia maio enviar encomenda cobrança entregar di...,0,2
20062,ctt - encomenda entregue à pessoa errada Fiz ...,ctt encomenda entregar pessoa errar fiz encome...,0,3
281,"Bom dia, gostaria de saber como podem resolver...",bom dia gostar saber poder resolver questão en...,0,3
17324,"ctt - encomenda entregue a outra pessoa, noutr...",ctt encomenda entregar outro pessoa noutro mor...,3,0


In [141]:
print('MISCLASSIFIED XGB TEXTS')
for i in range(100):
    print(xgb_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(xgb_misclassified_texts['true label'].iloc[i],
                                                                xgb_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED XGB TEXTS
ctt - serviço siga  Recebi o vale postal para levantamento de uma encomenda no passado dia 4 de Maio. Não tendo disponibilidade para levantar no posto CTT em questão, recorri ao serviço SIGA através do site dos CTT de forma a encaminhar a encomenda para o posto CTT do Porto (Município). Este pedido foi feito no dia 7 pelas 10h. Desde este dia, já me desloquei por duas vezes ao posto dos CTT do Porto sem que eles a tenham rececionado e desconhecendo o paradeiro da encomenda. O contato telefónico central dos CTT (com o qual só consegui falar 2 dias após a primeira tentativa) indicou-me para efetuar uma reclamação para o e-mail Já o fiz no dia 11 e, até hoje, também não obtive qualquer resposta. Neste momento desconheço o paradeiro da encomenda e onde posso efetuar o seu levantamento. 
TRUE LABEL IS 2 BUT IT WAS CLASSIFIED AS 0


ctt - os ctt dão-se ao luxo de brincar com os clientes!  Tenho uma encomenda de um dispositivo eletrónico vindo da China, que chegou a Li

### Apply classifier to the excluded class

In [142]:
xgb_predictions_excluded

,narrative_bert_title,narrative_tfidf_title,reason,initial Label
1,ctt - carta com certificado rasgado! - Boa No...,ctt carta certificar rasgar bom noite venho me...,2,1
19,"ctt - encomendas no chão Bom dia, é inadmissí...",ctt encomenda chão bom dia inadmissível cartei...,0,1
24,Fiz uma compra online em que viria uma encomen...,fiz compra online vir encomenda país enviar no...,2,1
26,ctt - encomenda não entregue Foi me enviada u...,ctt encomenda entregar enviar encomenda passad...,0,1
36,ctt - envio indevido para apresentação à alfân...,ctt envio indever apresentação alfândegar exmo...,0,1
...,...,...,...,...
22355,ctt - encomenda não entregue Venho por este m...,ctt encomenda entregar venho meio fazer reclam...,2,1
22359,ctt - cobrança das taxas já pagas no site Bom...,ctt cobrança taxa pagas site bom tarde dia fiz...,2,1
22368,ctt - tentativa falhada de levantar a minha en...,ctt tentativa falhada levantar encomenda bom n...,0,1
22370,fiz uma compra pela Internet na qual existia 3...,fiz compra Internet existir artigo chegar eu o...,2,1


In [143]:
print('PREDICTIONS OF XGB ON EXCLUDED TEXTS')
for i in range(100):
    print(xgb_predictions_excluded['narrative_bert_title'].iloc[i])
    print('INITIAL LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(xgb_predictions_excluded['initial Label'].iloc[i],
                                                                xgb_predictions_excluded['reason'].iloc[i]))
    print('\n')

PREDICTIONS OF XGB ON EXCLUDED TEXTS
ctt - carta com certificado rasgado!  - Boa Noite! - Venho por este meio reclamar que na data 31/08/2021 ás 10:00h, durante a passagem do correio deparei-me com uma carta que continha um documento de extrema importância (certificado profissional) com um rasgo. - Dito isto, este documento no seu estado não tem valor. - Obrigada! - Com os maiores cumprimentos: Lilia Pimentel 
INITIAL LABEL IS 1 BUT IT WAS CLASSIFIED AS 2


ctt - encomendas no chão  Bom dia, é inadmissível que o carteiro em vez de colocar as encomendas na caixa de correio as mande para dentro de nossa casa como se de um jornal se tratasse e vivêssemos num país americano! Para além de estar um dia de muita chuva onde danificou bastante as mesmas o meu cão anda pela casa mas quando vê objetos novos brinca com os mesmos e os destrói como foi o caso. E o carteiro que não me venha dizer que não conseguia colocar na caixa porque quando isso acontece deixasse um papel para ir levantar a encom

### Use predicted classes as new labels to train new classifier

In [144]:
# Concatenate the excluded predictions dataframe with the original dataset, keeping only the specified columns
new_dataset = pd.concat([xgb_filtered_df[['reason', 'narrative_tfidf_title','narrative_bert_title']], 
                         xgb_predictions_excluded[['reason', 'narrative_tfidf_title','narrative_bert_title']]], ignore_index=True)


shuffled_dataset = new_dataset.sample(frac=1, random_state=42)
shuffled_dataset

,reason,narrative_tfidf_title,narrative_bert_title
6769,0,ctt reclamação dia dezembro enviar encomenda c...,ctt - reclamação No dia 9 de dezembro enviei ...
8171,2,ctt atraso correspondência reformar doença sem...,ctt - atraso da correspondência Sou reformada...
6603,0,ctt reclamação carteiro venho poe meio reclama...,ctt - reclamação para com o carteiro Venho po...
5305,0,ctt anulação pagar resolvido fiz envio estrang...,"ctt - anulação paga, mas não resolvida Fiz um..."
20251,0,ctt problema receber encomenda sinceramente nu...,ctt - problemas em receber minhas encomendas ...
...,...,...,...
11964,0,ctt irresponsabilidade negligência falta respe...,"ctt - irresponsabilidade, negligência e falta ..."
21575,0,ctt correspondência abrir venho meio apresenta...,ctt - correspondência aberta Venho por este m...
5390,2,ctt alfândegar féria dar entrada dia julho alf...,ctt - alfândega de férias Deu entrada no dia ...
860,2,ctt remessa pedir desalfandegamento recebi pre...,ctt - remessa com pedido de desalfandegamento ...


In [149]:
new_classifier(shuffled_dataset, XGBClassifier(), 1)

              precision    recall  f1-score   support

           0       0.70      0.67      0.68      1402
           2       0.75      0.83      0.79      1720
           3       0.58      0.26      0.36       242

    accuracy                           0.72      3364
   macro avg       0.68      0.59      0.61      3364
weighted avg       0.72      0.72      0.71      3364

[[ 943  423   36]
 [ 281 1430    9]
 [ 132   48   62]]


# SVC

###  Misclassified texts

In [150]:
svc_predictions, svc_true_labels, svc_misclassified_texts, svc_predictions_excluded, svc_filtered_df= remove_class_and_train_classifier(df, 1,LinearSVC())

              precision    recall  f1-score   support

           0       0.68      0.69      0.68      1195
           2       0.78      0.84      0.81      1609
           3       0.62      0.25      0.35       228

    accuracy                           0.73      3032
   macro avg       0.69      0.59      0.61      3032
weighted avg       0.73      0.73      0.72      3032

[[ 821  343   31]
 [ 258 1347    4]
 [ 127   45   56]]


In [151]:
svc_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
19984,"ctt - vale correio extraviado Ola, supostamen...",ctt valer correio extraviar ola supostamente s...,2,0
7448,"Boa tarde, venho por este meio informar a negl...",bom tarde venho meio informar negligencia part...,3,0
7340,ctt - os ctt dão-se ao luxo de brincar com os ...,ctt ctt luxo brincar cliente encomenda disposi...,2,0
18728,"ctt - não entrega cartas Boa tarde, Venho por...",ctt entregar carta bom tarde venho meio morosi...,2,0
4967,Fiz uma compra online já com o pagamento do Iv...,fiz compra online pagamento iva ato compra fat...,0,2
...,...,...,...,...
18508,Dia 23 Maio foi enviado uma encomenda a cobran...,dia maio enviar encomenda cobrança entregar di...,0,2
20062,ctt - encomenda entregue à pessoa errada Fiz ...,ctt encomenda entregar pessoa errar fiz encome...,0,3
281,"Bom dia, gostaria de saber como podem resolver...",bom dia gostar saber poder resolver questão en...,0,3
17324,"ctt - encomenda entregue a outra pessoa, noutr...",ctt encomenda entregar outro pessoa noutro mor...,3,0


In [153]:
print('MISCLASSIFIED SVC TEXTS')
for i in range(100):
    print(svc_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(svc_misclassified_texts['true label'].iloc[i],
                                                                svc_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED SVC TEXTS
ctt - vale correio extraviado  Ola, supostamente a segurança social enviou um vale correio para a minha morada e passados 2 meses nada chegou e ninguém sabe para onde o cheque foi parar.fui fazer uma reclamação e ainda me pediram 2,50 para enviar a reclamação de um serviço mal prestado pela parte dos mesmos.tentei resolver a situação onde não sabem onde está o vale e pedem me informações de um vale que eu nem sequer vi e não poderei saber tais informações 
TRUE LABEL IS 0 BUT IT WAS CLASSIFIED AS 2


Boa tarde, venho por este meio informar a negligencia por parte dos colaboradores do ctt correios de delães/ Vila nova de Famalicão, a má entrega das cartas das respetivas pessoas. Não sendo a primeira vez que me desloco ao balcão da mesma para entrega de cartas as quais não me são pretensidas. Informo que a minha caixa de correio se encontra respetivamente identificada com o meu nome e o da minha esposa, é não com o nome de terceiros. Sem mais nada acrescentar agra

### Apply classifier to the excluded class

In [152]:
svc_predictions_excluded

,narrative_bert_title,narrative_tfidf_title,reason,initial Label
1,ctt - carta com certificado rasgado! - Boa No...,ctt carta certificar rasgar bom noite venho me...,0,1
19,"ctt - encomendas no chão Bom dia, é inadmissí...",ctt encomenda chão bom dia inadmissível cartei...,0,1
24,Fiz uma compra online em que viria uma encomen...,fiz compra online vir encomenda país enviar no...,2,1
26,ctt - encomenda não entregue Foi me enviada u...,ctt encomenda entregar enviar encomenda passad...,0,1
36,ctt - envio indevido para apresentação à alfân...,ctt envio indever apresentação alfândegar exmo...,0,1
...,...,...,...,...
22355,ctt - encomenda não entregue Venho por este m...,ctt encomenda entregar venho meio fazer reclam...,2,1
22359,ctt - cobrança das taxas já pagas no site Bom...,ctt cobrança taxa pagas site bom tarde dia fiz...,2,1
22368,ctt - tentativa falhada de levantar a minha en...,ctt tentativa falhada levantar encomenda bom n...,0,1
22370,fiz uma compra pela Internet na qual existia 3...,fiz compra Internet existir artigo chegar eu o...,2,1


In [154]:
print('PREDICTIONS OF SVC ON EXCLUDED TEXTS')
for i in range(100):
    print(svc_predictions_excluded['narrative_bert_title'].iloc[i])
    print('INITIAL LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(svc_predictions_excluded['initial Label'].iloc[i],
                                                                svc_predictions_excluded['reason'].iloc[i]))
    print('\n')

PREDICTIONS OF SVC ON EXCLUDED TEXTS
ctt - carta com certificado rasgado!  - Boa Noite! - Venho por este meio reclamar que na data 31/08/2021 ás 10:00h, durante a passagem do correio deparei-me com uma carta que continha um documento de extrema importância (certificado profissional) com um rasgo. - Dito isto, este documento no seu estado não tem valor. - Obrigada! - Com os maiores cumprimentos: Lilia Pimentel 
INITIAL LABEL IS 1 BUT IT WAS CLASSIFIED AS 0


ctt - encomendas no chão  Bom dia, é inadmissível que o carteiro em vez de colocar as encomendas na caixa de correio as mande para dentro de nossa casa como se de um jornal se tratasse e vivêssemos num país americano! Para além de estar um dia de muita chuva onde danificou bastante as mesmas o meu cão anda pela casa mas quando vê objetos novos brinca com os mesmos e os destrói como foi o caso. E o carteiro que não me venha dizer que não conseguia colocar na caixa porque quando isso acontece deixasse um papel para ir levantar a encom

### Use predicted classes as new labels to train new classifier

In [155]:
# Concatenate the excluded predictions dataframe with the original dataset, keeping only the specified columns
new_dataset = pd.concat([svc_filtered_df[['reason', 'narrative_tfidf_title','narrative_bert_title']], 
                         svc_predictions_excluded[['reason', 'narrative_tfidf_title','narrative_bert_title']]], ignore_index=True)


shuffled_dataset = new_dataset.sample(frac=1, random_state=42)
shuffled_dataset

,reason,narrative_tfidf_title,narrative_bert_title
6769,0,ctt reclamação dia dezembro enviar encomenda c...,ctt - reclamação No dia 9 de dezembro enviei ...
8171,2,ctt atraso correspondência reformar doença sem...,ctt - atraso da correspondência Sou reformada...
6603,0,ctt reclamação carteiro venho poe meio reclama...,ctt - reclamação para com o carteiro Venho po...
5305,0,ctt anulação pagar resolvido fiz envio estrang...,"ctt - anulação paga, mas não resolvida Fiz um..."
20251,0,ctt problema receber encomenda sinceramente nu...,ctt - problemas em receber minhas encomendas ...
...,...,...,...
11964,0,ctt irresponsabilidade negligência falta respe...,"ctt - irresponsabilidade, negligência e falta ..."
21575,0,ctt correspondência abrir venho meio apresenta...,ctt - correspondência aberta Venho por este m...
5390,2,ctt alfândegar féria dar entrada dia julho alf...,ctt - alfândega de férias Deu entrada no dia ...
860,2,ctt remessa pedir desalfandegamento recebi pre...,ctt - remessa com pedido de desalfandegamento ...


In [156]:
new_classifier(shuffled_dataset, LinearSVC(), 1)

              precision    recall  f1-score   support

           0       0.71      0.74      0.73      1412
           2       0.78      0.82      0.80      1705
           3       0.66      0.28      0.39       247

    accuracy                           0.75      3364
   macro avg       0.72      0.61      0.64      3364
weighted avg       0.74      0.75      0.74      3364

[[1043  345   24]
 [ 288 1406   11]
 [ 129   49   69]]
